# 라이브러리 import와  크롬 자동실행

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}

In [2]:
path = './chromedriver.exe'
browser = webdriver.Chrome(path)

url = 'https://www.naver.com/'
browser.get(url)

### 로그인 - 보안기능땜에 수동로그인 해줘야

In [3]:
import json
with open('./account.json', encoding='UTF8') as f:
    account=json.load(f)

In [4]:
url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
browser.get(url)

In [5]:
browser.find_element_by_css_selector('#id').clear()
browser.find_element_by_css_selector('#pw').clear()
browser.find_element_by_css_selector('#id').send_keys(account['id'])
browser.find_element_by_css_selector('#pw').send_keys(account['pw'])

In [6]:
browser.find_element_by_class_name('btn_global').click()

## selenium 활용해서 스팸메일 읽어오기

In [7]:
browser.get('https://mail.naver.com/')

In [8]:
browser.find_element_by_css_selector('span.item_wrap.bu6').click()

In [9]:
from collections import defaultdict

spam_list = defaultdict()
idx=0
for tag in browser.find_elements_by_css_selector('strong.mail_title'):
    print(tag.text)
    spam_list[idx]=tag.text
    idx+=1
spam_list[0]

(광고) [멸치쇼핑] 고민없이 오늘도 멸치쇼핑!
(광고) 창업 때문에 고민하신다면 추천해 드립니다.
(광고) 최승호님께 드리는 휴식같은 선물! 주말에만 드려요
[고양시 덕양구] 검색 결과 알림ㅣ최승호님을 위한 채용소식 입니다
2년 공시준비하다가 면탈하니 아쉬움밖에 안남네요... / 현재 30인데 25살부터 생각해오던 공무원을 해보려합니다.
♨면접관의_거짓말_1위는? #교직원현직자_솔직인터뷰 #취준생토익평균점수_확인↓ #합동채용선호기업1위
(광고) 폼클렌징 단돈 1,900원 +무료배송
(광고) 상품 하나만 구매해도 사은품 증정 드려요! 오늘 밤 12시 종료!
(광고)100대 한정★6세대 맥북에어Style 울트라북 379,000원
[즉시 지원] 최근 최승호님 활동으로 ㈜카스 채용 공고가 매칭되었습니다.
(광고) 키친아트 후라이팬 1+1 13,900원 +무배
최승호님, 에이치엘그린파워(주) 등 오늘의 공채 소식입니다
★한국전력공사/기아자동차 채용 시작!★& 무료 가이드북 배포중!
(광고) 가장 인기 있는 BEST 상품만 엄선했습니다!
♨한국가스안전공사채용형인턴_122명 #한국전력공사64명_채용형인턴 #기술보증기금_인턴_51명 #NH투자증권_신입
(광고) 간편식/술안주 단돈 790원 +무배+무제한적립
네이버 클라우드 플랫폼 서비스 이용약관 개정 안내
직원복지 甲, 멤버쉽카드·복지포인트
최승호님을 위한 추천 채용공고
[고양시 덕양구] 검색 결과 알림ㅣ최승호님을 위한 채용소식 입니다
국방부 군무원 평균 경쟁률 ‘34.8대 1’ / 2019년 교육청 9급 경쟁률 전년대비 '하락'
(광고) USB 미니 선풍기 2,900원 균일가 +무배
Case# 90321528: Gnip Freemium Access Request [ref:00DA0000000K0A8.5004A00001SmGGN:ref]
@dtmedrNv0olkGkw 님, 하고픈 말을 해보세요.
(광고) 리얼 아이스 망고바 10개 8,900원 +무배
[즉시 지원] 최근 최승호님 활동으로 시몬스 채용 공고가 매칭되었습니다.


'(광고) [멸치쇼핑] 고민없이 오늘도 멸치쇼핑!'

# bs를 활용해서 일반 메일 가져오기

In [12]:
browser.get('https://mail.naver.com')
html=browser.page_source
dom = BeautifulSoup(html, 'lxml')

In [13]:
correct_list=list()

correct_list=[_.text.strip()[6:] for _ in dom.select('strong.mail_title') ]

print(correct_list[1])
print(correct_list[3])
print(correct_list[7])

매월 개설! SK 노하우만 꽉꽉 담은 빅데이터 핸즈온 세미나에 초대합니다.
안녕하세요. 365MC 의료 IT ( AI / App 개발 ) 인턴 프로그램 담당자 입니다.
NHN KCP - 무신사스토어의 결제 내역입니다.


# NaiveBayes를 이용한 스팸분류기!

In [14]:
from math import log2
from konlpy.tag import Komoran
from math import sqrt

In [15]:
correct_list[1]

'매월 개설! SK 노하우만 꽉꽉 담은 빅데이터 핸즈온 세미나에 초대합니다.'

In [16]:
ma = Komoran()

DTM = defaultdict(lambda:defaultdict(int))

for term in correct_list:
    i=0
    for token in term.split():
        DTM[i][token] += 1
        i += 1    
print(DTM[3].keys())

dict_keys(['데이터분석', '노하우만', '안내', 'IT', 'Internship', '인턴십', '문제', '무신사스토어의', '완료되었습니다', '최승호님', '㈜마크로젠', '오늘의', '공고현황을', 'United,', '기업정보를', '이력서를', '공고도', '유사한', '되었습니다.', '성공적으로', '시뮬레이션', '배송이'])


In [17]:
spam_list

defaultdict(None,
            {0: '(광고) [멸치쇼핑] 고민없이 오늘도 멸치쇼핑!',
             1: '(광고) 창업 때문에 고민하신다면 추천해 드립니다.',
             2: '(광고) 최승호님께 드리는 휴식같은 선물! 주말에만 드려요',
             3: '[고양시 덕양구] 검색 결과 알림ㅣ최승호님을 위한 채용소식 입니다',
             4: '2년 공시준비하다가 면탈하니 아쉬움밖에 안남네요... / 현재 30인데 25살부터 생각해오던 공무원을 해보려합니다.',
             5: '♨면접관의_거짓말_1위는? #교직원현직자_솔직인터뷰 #취준생토익평균점수_확인↓ #합동채용선호기업1위',
             6: '(광고) 폼클렌징 단돈 1,900원 +무료배송',
             7: '(광고) 상품 하나만 구매해도 사은품 증정 드려요! 오늘 밤 12시 종료!',
             8: '(광고)100대 한정★6세대 맥북에어Style 울트라북 379,000원',
             9: '[즉시 지원] 최근 최승호님 활동으로 ㈜카스 채용 공고가 매칭되었습니다.',
             10: '(광고) 키친아트 후라이팬 1+1 13,900원 +무배',
             11: '최승호님, 에이치엘그린파워(주) 등 오늘의 공채 소식입니다',
             12: '★한국전력공사/기아자동차 채용 시작!★& 무료 가이드북 배포중!',
             13: '(광고) 가장 인기 있는 BEST 상품만 엄선했습니다!',
             14: '♨한국가스안전공사채용형인턴_122명 #한국전력공사64명_채용형인턴 #기술보증기금_인턴_51명 #NH투자증권_신입',
             15: '(광고) 간편식/술안주 단돈 790원 +무배+무제한적립',
             16: '네이버 클라우드 플랫폼 서비스 이용약관 개정 안내',
   

In [18]:
trueData = [(correct_list.index(_), _, True) for _ in correct_list]
falseData = [(len(trueData) + _, spam_list[_], False)for _ in spam_list]

In [27]:
trainingData = trueData + falseData
trainingData[7:10]
testingData = [term[0] for term in ma.pos(correct_list[4]) if term[1].startswith('N')]

In [28]:
V = list(set([term[0] for _ in trainingData for term in ma.pos(_[1]) if term[1].startswith('N')]))
N = len(testingData)

In [29]:
from collections import defaultdict

prior = list()

        
Tct = defaultdict(int)
for data in trueData:  
    Nc = len(trueData)
    PriorC = Nc/N
    for term in data[1].lower().split():
        Tct[term] += 1

_Tct = defaultdict(int)
for data in falseData:  
    _Nc = len(falseData)
    _PriorC = _Nc/N
    for term in data[1].lower().split():
        _Tct[term] += 1

In [30]:
condProbC = defaultdict(float)
_condProbC = defaultdict(float)

countSum = sum(Tct.values())
_countSum = sum(_Tct.values())

for term, freq in Tct.items():
    condProbC[term] = (freq+1)/(countSum+len(V))
    
for term, freq in _Tct.items():
    _condProbC[term] = (freq+1)/(_countSum+len(V))  

In [31]:
# P(C)multi(P(Tct|C)) ===> log(P(C)) + SUM(P(Tct|C))
from math import log, exp

# Prior Probability
result = log(PriorC)
_result = log(_PriorC)

# joint probability => conditional independence
for term in testingData[1].lower().split():
    # multi -> log sum
    result += log((Tct[term]+1)/(countSum+len(V)))
    _result += log((_Tct[term]+1)/(_countSum+len(V)))

# optimal classfier
if result > _result:
    print("정상메일입니다.", result, exp(result))
else:
    print("스팸메일입니다.", _result, exp(_result))
result, _result

정상메일입니다. -3.00071981506503 0.04975124378109454


(-3.00071981506503, -3.7447870860522317)